In [4]:
import pandas as pd
import numpy as np
from datetime import datetime
import scipy as scipy

In [5]:
from scipy.stats import poisson

In [6]:
from scipy import optimize

In [7]:
sf = pd.read_csv('../data/raw_data/san_francisco.csv', low_memory=False)

In [8]:
#sj = pd.read_csv('../data/raw_data/san_jose.csv', low_memory=False)

In [9]:
#sj

In [10]:
stop_by_race = sf["subject_race"].value_counts()

In [11]:
stop_by_race

white                     372318
asian/pacific islander    157684
black                     152196
hispanic                  116014
other                     106858
Name: subject_race, dtype: int64

In [12]:
race_index = ['white', 'asian/pacific islander', 'black', 'hispanic', 'other']
# http://www.bayareacensus.ca.gov/bayarea.htm

In [13]:
sf_pop_list = [.402, .36+.005, .056, .152, .007+.045
              ] # https://www.census.gov/quickfacts/sanfranciscocountycalifornia

In [14]:
ba_pop_list = [.424, 0.230+0.003, 0.064, 0.235, 0.038 ]

In [15]:
sf_pop = pd.Series(sf_pop_list, index = race_index)

In [16]:
ba_pop = pd.Series(ba_pop_list, index = race_index)

In [17]:
sf_pop

white                     0.402
asian/pacific islander    0.365
black                     0.056
hispanic                  0.152
other                     0.052
dtype: float64

In [18]:
pop_by_race = round(881549*sf_pop)

In [19]:
pop_by_race

white                     354383.0
asian/pacific islander    321765.0
black                      49367.0
hispanic                  133995.0
other                      45841.0
dtype: float64

In [20]:
ba_pop_by_race = round(7150739 * ba_pop)

In [21]:
ba_pop_by_race

white                     3031913.0
asian/pacific islander    1666122.0
black                      457647.0
hispanic                  1680424.0
other                      271728.0
dtype: float64

The Bayesian Information Criterion is a scoring mechanism that can be used to compare models with different numbers of parameters. The model with the lowest BIC should be favored. There is plenty of debate about quantifying *how much* it should be favored. A difference of greater than 10 is pretty clear, though.

In [22]:
def bic(params, data, minus_max_like):
    result = len(params)*np.log(data.count()) + 2*minus_max_like
    return result

Model 1: The police are unbiased, and stop all citizens at the same rate, regardless of race.

In [23]:
def ln_like_1(mu, stops, pops):
    lnl=0
    for idx in stop_by_race.index:
        expect = mu*pops[idx]
        pois = scipy.stats.poisson.pmf(stops[idx], expect)
        lnl = lnl + np.log(pois)
        print("{}: expect {}, got {}. Prob: {}".format(idx, expect, stops[idx], pois))
        
    return lnl

I was not able to optimize for Model 1, because I couldn't find any single value of the stop rate that had a likelihood large enough to hold in a float. They were all -Inf, because they were so unlikely. I think we can rule out Model 1.

Model 2: The police are unbiased, and stop all citizens at the same rate, regardless of race. However, the city population is a bad proxy for the people in the city at any given time; so we include a parameter for the fraction of the Bay Area population, by race, that is in SF on any given day.

In [24]:
def ln_like_2(mu, stops, pops, transient):
    lnl=0
    for idx in stop_by_race.index:
        expect = mu*(pops[idx] + transient[idx])
        pois = scipy.stats.poisson.pmf(stops[idx], expect)
        if(pois <= 0):
            pois = 1e-180
        lnl = lnl + np.log(pois)
        #print("{}: expect {}, got {}. Prob: {}".format(idx, expect, stops[idx], pois))
        
    return lnl

The optimizer needs the parameters passed in specific ways, and minimizes rather than maximizes. This function is just a wrapper.

In [25]:
def ln_like_opt(x0, stop_by_race, pop_by_race, ba_pop_by_race):
    return -ln_like_2(x0[0], stop_by_race, pop_by_race, x0[1:] * ba_pop_by_race)

The parameters are the Stop Rate for everyone, and then one parameter for each race designating the fraction of the Bay Area population that must have been in the city to explain the data.

In [26]:
initial = [0.48, 0.14, 0, 0.58, 0.06, 0.7]

In [27]:
ln_like_opt(initial, stop_by_race, pop_by_race, ba_pop_by_race)

310.1328243294993

In [28]:
res = optimize.minimize(ln_like_opt, args=(stop_by_race, pop_by_race, ba_pop_by_race), 
                       x0=initial, method = 'Nelder-Mead', options={'maxiter':500})

In [29]:
res

 final_simplex: (array([[0.48702276, 0.1352592 , 0.00120557, 0.57498163, 0.06202079,
        0.63875725],
       [0.48700965, 0.13526957, 0.00120778, 0.57498887, 0.06201964,
        0.63878369],
       [0.48701569, 0.13526547, 0.00120472, 0.57500042, 0.0620189 ,
        0.63876835],
       [0.48701628, 0.13526054, 0.00120437, 0.57500253, 0.0620191 ,
        0.63876724],
       [0.48701947, 0.13526262, 0.00120422, 0.57499602, 0.06202133,
        0.63878215],
       [0.4870232 , 0.13525883, 0.00120564, 0.57499102, 0.06201688,
        0.63878661],
       [0.48702953, 0.13525826, 0.00120598, 0.57495466, 0.06201619,
        0.63877224]]), array([34.57948905, 34.57949662, 34.57951365, 34.57952611, 34.57953688,
       34.57954331, 34.57956223]))
           fun: 34.57948905394005
       message: 'Optimization terminated successfully.'
          nfev: 248
           nit: 150
        status: 0
       success: True
             x: array([0.48702276, 0.1352592 , 0.00120557, 0.57498163, 0.06202079,

In [30]:
minus_max_like_2 = ln_like_opt(res.x, stop_by_race, pop_by_race, ba_pop_by_race)

In [34]:
minus_max_like_2

34.57948905394005

The optimized likelihood is smaller than my initial guess! It better be, if the optimizer is doing its job.

In [33]:
print("Police stop people, on average, {:.2f} times per year.".format(res.x[0]))
for idx in range(5):
    print("{:.2f}% of {} people in the Bay Area must have been in SF each day".format(100*res.x[1+idx], race_index[idx]))

Police stop people, on average, 0.49 times per year.
13.53% of white people in the Bay Area must have been in SF each day
0.12% of asian/pacific islander people in the Bay Area must have been in SF each day
57.50% of black people in the Bay Area must have been in SF each day
6.20% of hispanic people in the Bay Area must have been in SF each day
63.88% of other people in the Bay Area must have been in SF each day


This commute pattern is what would be required for Model 1 to be correct. We have not explicitly included Bayesian priors, but these values seem pretty improbable.

In [31]:
bic(initial, stop_by_race, minus_max_like_2)

78.81560558248471

Model 3 allows different stop rates, and transient populations. However, these parameters are degenerate, so we can't expect much from this model. It will match Model 2, except with more parameters, so it's BIC will be higher.

In [35]:
def ln_like_3(mu, stops, pops, transient):
    lnl=0
    for idx in stop_by_race.index:
        expect = mu[idx]*(pops[idx] + transient[idx])
        pois = scipy.stats.poisson.pmf(stops[idx], expect)
        if(pois <= 0):
            pois = 1e-180
        lnl = lnl + np.log(pois)
        #print("{}: expect {}, got {}. Prob: {}".format(idx, expect, stops[idx], pois))
        
    return lnl

In [36]:
def ln_like_opt_3(x0, stop_by_race, pop_by_race, ba_pop_by_race):
    mu = pd.Series(x0[0:5], index = race_index)
    return -ln_like_3(mu, stop_by_race, pop_by_race, x0[5:] * ba_pop_by_race)

In [37]:
initial_3 = (stop_by_race/pop_by_race).values

In [38]:
initial_3 = np.append(initial_3, [0, 0, 0, 0, 0])

In [39]:
res = optimize.minimize(ln_like_opt_3, args=(stop_by_race, pop_by_race, ba_pop_by_race), 
                       x0=initial_3, method = 'Nelder-Mead', options={'maxiter':500})

In [40]:
res

 final_simplex: (array([[1.05053872e+00, 4.90037165e-01, 3.08265317e+00, 8.65717390e-01,
        2.33088215e+00, 7.82782795e-06, 8.81393923e-06, 1.03929019e-05,
        8.38435166e-06, 1.26845293e-05],
       [1.05053231e+00, 4.90035134e-01, 3.08262609e+00, 8.65709093e-01,
        2.33086618e+00, 8.54225081e-06, 9.61431542e-06, 1.13418217e-05,
        9.14850812e-06, 1.38362220e-05],
       [1.05053555e+00, 4.90036148e-01, 3.08263976e+00, 8.65713270e-01,
        2.33087422e+00, 8.18333460e-06, 9.21438759e-06, 1.08652695e-05,
        8.76456481e-06, 1.32561109e-05],
       [1.05053083e+00, 4.90034660e-01, 3.08261989e+00, 8.65707191e-01,
        2.33086250e+00, 8.70569778e-06, 9.79882075e-06, 1.15573920e-05,
        9.32576828e-06, 1.41027069e-05],
       [1.05054009e+00, 4.90037596e-01, 3.08265892e+00, 8.65719138e-01,
        2.33088544e+00, 7.67594261e-06, 8.64437364e-06, 1.01927390e-05,
        8.22305060e-06, 1.24411244e-05],
       [1.05053272e+00, 4.90035253e-01, 3.08262781e+00, 8.

In [41]:
minus_max_like_3 = ln_like_opt_3(res.x, stop_by_race, pop_by_race, ba_pop)

In [42]:
bic(initial_3, stop_by_race, minus_max_like_3)

85.25856560948182

Model 4: Police stop people at different rates by race, but also some fraction of the Bay Area is in the City each day. However, we assume that the people in the city for the day match the demographics of the Bay Area.

In [43]:
def ln_like_opt_4(x0, stop_by_race, pop_by_race, ba_pop_by_race):
    mu = pd.Series(x0[0:5], index = race_index)
    return -ln_like_3(mu, stop_by_race, pop_by_race, x0[5] * ba_pop_by_race)

In [44]:
initial_4 = np.append((stop_by_race/pop_by_race).values, 0.05)

In [45]:
res = optimize.minimize(ln_like_opt_4, args=(stop_by_race, pop_by_race, ba_pop_by_race), 
                       x0=initial_4, method = 'Nelder-Mead', options={'maxiter':500})

In [46]:
res

 final_simplex: (array([[1.05060909, 0.49005952, 3.08295015, 0.86580843, 2.33105735,
        0.05      ],
       [1.05063474, 0.49005952, 3.08295015, 0.86580843, 2.33105735,
        0.05      ],
       [1.05060909, 0.49007148, 3.08295015, 0.86580843, 2.33105735,
        0.05      ],
       [1.05060909, 0.49005952, 3.08302542, 0.86580843, 2.33105735,
        0.05      ],
       [1.05060909, 0.49005952, 3.08295015, 0.86582956, 2.33105735,
        0.05      ],
       [1.05060909, 0.49005952, 3.08295015, 0.86580843, 2.33111426,
        0.05      ],
       [1.05060909, 0.49005952, 3.08295015, 0.86580843, 2.33105735,
        0.05000122]]), array([2072.32658369, 2072.32658369, 2072.32658369, 2072.32658369,
       2072.32658369, 2072.32658369, 2072.32658369]))
           fun: 2072.326583694641
       message: 'Optimization terminated successfully.'
          nfev: 95
           nit: 12
        status: 0
       success: True
             x: array([1.05060909, 0.49005952, 3.08295015, 0.86580843,

In [47]:
ln_like_opt_4(res.x, stop_by_race, pop_by_race, ba_pop)

34.57944478651916

In [48]:
stop_by_race.count()

5

In [49]:
bic(res.x, stop_by_race, ln_like_opt_4(res.x, stop_by_race, pop_by_race, ba_pop))

78.81551704764293

Again, this is a semi-degenerate case, so we don't gain enough to justify extra parameters.

Here's the inkling of a different idea: can we learn things by comparing Moving Violations to Non-moving Violations? 

In [63]:
sf[sf["subject_race"]=="black"]["reason_for_stop"].value_counts()

Moving Violation                                              75450
Mechanical or Non-Moving Violation (V.C.)                     73479
MPC Violation                                                   930
BOLO/APB/Warrant                                                646
Assistance to Motorist                                          240
DUI Check                                                       188
Traffic Collision                                               169
Moving Violation|Mechanical or Non-Moving Violation (V.C.)       29
Moving Violation|MPC Violation                                    2
Moving Violation|BOLO/APB/Warrant                                 1
Name: reason_for_stop, dtype: int64

In [64]:
sf[sf["subject_race"]=="white"]["reason_for_stop"].value_counts()

Moving Violation                                                                                        244182
Mechanical or Non-Moving Violation (V.C.)                                                               125184
MPC Violation                                                                                             1004
Traffic Collision                                                                                          475
DUI Check                                                                                                  395
Assistance to Motorist                                                                                     329
BOLO/APB/Warrant                                                                                           208
Moving Violation|Mechanical or Non-Moving Violation (V.C.)                                                  74
Moving Violation|MPC Violation                                                                               4
M